# DDQN

In [1]:
import matplotlib
import random

from collections import deque, namedtuple
from itertools import count
from time import time

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as utils
import torch.optim as optim
import pandas as pd

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    pass

In [2]:
torch.cuda.is_available()

True

In [3]:
device_name = "cpu"

if True:
    if torch.cuda.is_available():
        device_name = "cuda"
    elif torch.mps.is_available():
        device_name = "mps"
    # elif torch.hip.is_available():
    #     device_name = "hip"
    elif torch.mtia.is_available():
        device_name = "mtia"
    elif torch.xpu.is_available():
        device_name = "xpu"

device = torch.device(device_name)
print(f"device: {device_name}")

device: cuda


In [4]:
UPDATE_TARGET_STEPS = 500
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000000
TAU = 0.005
LR = 1e-4

In [5]:
import sys
sys.path.append(r"C:\Users\takat\PycharmProjects\machine-learning")
import flowdata
import flowenv

In [6]:
# from flowdata.src.flow_data import using_data

raw_data_train, raw_data_test = flowdata.flow_data.using_data()

train_env = gym.make("flowenv/FlowTrain-v0", data=raw_data_train)
test_env = gym.make("flowenv/FlowTest-v0", data=raw_data_test)

In [7]:
Transaction = namedtuple('Transaction', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        # self.capacity = capacity
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transaction(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [8]:
class DQN(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(n_inputs, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, n_outputs)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [9]:
def select_action(state):
    global n_actions, policy_net, steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1

    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], dtype=torch.long, device=device)

def optimize_model():
    global policy_net, target_net, optimizer, memory

    if len(memory) < BATCH_SIZE:
        return

    transitions = memory.sample(BATCH_SIZE)
    batch = Transaction(*zip(*transitions))

    non_final_mask = torch.tensor(
        tuple(map(lambda s: s is not None, batch.next_state)),
        device=device,
        dtype=torch.bool
    )
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_actions = policy_net(non_final_next_states).max(1).indices.unsqueeze(1)
        next_state_values[non_final_mask] = target_net(non_final_next_states).gather(1, next_actions).squeeze(1)

    expected_state_action_values = reward_batch + GAMMA * next_state_values

    criterion = nn.MSELoss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

def get_h_m_s(seconds: float):
    hours = int(seconds // 3600)
    minutes = int((seconds - hours * 3600) // 60)
    seconds = seconds - hours * 3600 - minutes * 60
    return hours, minutes, seconds

def loading_bar(episode, total_episodes, interval):
    pro_size_float = (episode + 1) / total_episodes * 20
    show = pro_size_float * 5
    pro_size = int(pro_size_float)

    # episode...interval -> total_episodes...interval * total_episodes / episode
    last_time = interval * (total_episodes - episode) / (episode + 1)
    hours, minutes, seconds = get_h_m_s(last_time)
    print(f"\r[{'#' * pro_size}{' ' * (20 - pro_size)}] {show:3.02f}%, last={hours:02d}:{minutes:02d}:{seconds:03.3f}", end="")

In [ ]:
n_actions = train_env.action_space.n
n_inputs = train_env.observation_space.shape[0]

state, info = train_env.reset()

policy_net = DQN(n_inputs, n_actions).to(device)
target_net = DQN(n_inputs, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.Adam(policy_net.parameters(), lr=LR)
memory = ReplayMemory(int(1e6))

num_episodes = 10000
steps_done = 0
episode_rewards = []
mean_rewards = []

x = []

start_time = time()
for i_episode in range(num_episodes):
    state, info = train_env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

    sum_reward = 0

    for t in count():
        action = select_action(state)
        next_state, reward, terminated, truncated, _ = train_env.step(action.item())

        reward = torch.tensor([reward], device=device)
        done = bool(terminated)

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0)

        memory.push(state, action, next_state, reward)

        state = next_state
        sum_reward += reward.item()

        optimize_model()

        if steps_done % UPDATE_TARGET_STEPS == 0:
            target_net_state_dict = target_net.state_dict()
            policy_net_state_dict = policy_net.state_dict()
            for key in target_net_state_dict:
                target_net_state_dict[key] = TAU * policy_net_state_dict[key] + (1 - TAU) * target_net_state_dict[key]
            target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_rewards.append(sum_reward)
            mean_rewards.append(np.mean(episode_rewards))
            break
    x.append(i_episode)
    end_time = time()

    if i_episode > 0 and i_episode % 100 == 0:
        print(f" Episode {i_episode}, reward: {sum_reward}")

    loading_bar(i_episode, num_episodes, end_time - start_time)

plt.plot(episode_rewards)
plt.plot(mean_rewards, color="red")
plt.show()

result = pd.DataFrame(episode_rewards, columns=["reward"])
torch.save(policy_net.state_dict(), "ddqn.pth")
train_env.close()

[                    ] 1.00%, last=01:27:27.170 Episode 100, reward: 16
[                    ] 2.00%, last=01:27:33.330 Episode 200, reward: 12
[                    ] 3.00%, last=01:27:9.3073 Episode 300, reward: 4
[                    ] 4.00%, last=01:26:27.363 Episode 400, reward: 22
[#                   ] 5.00%, last=01:26:3.6689 Episode 500, reward: 10
[#                   ] 6.00%, last=01:25:36.058 Episode 600, reward: 10
[#                   ] 7.00%, last=01:24:51.765 Episode 700, reward: 4
[#                   ] 8.00%, last=01:24:19.680 Episode 800, reward: 22
[#                   ] 9.00%, last=01:23:53.786 Episode 900, reward: 10
[##                  ] 10.00%, last=01:23:23.886 Episode 1000, reward: 8
[##                  ] 11.00%, last=01:23:0.8740 Episode 1100, reward: 12
[##                  ] 12.00%, last=01:22:14.062 Episode 1200, reward: 28
[##                  ] 13.00%, last=01:21:32.716 Episode 1300, reward: 22
[##                  ] 14.00%, last=01:21:3.6466 Episode 14

In [ ]:
mean_rewards = []

for i in range(0, len(episode_rewards)):
    # print(f"Episode {i}, mean reward: {np.mean(episode_rewards[0:i])}")
    mean_rewards.append(np.mean(episode_rewards[0:i]))

plt.figure(figsize=(10, 5))
plt.plot(episode_rewards)
plt.plot(mean_rewards, color="red")
plt.show()

In [ ]:
trained_net = DQN(n_inputs, n_actions).to(device)
trained_net.load_state_dict(torch.load("ddqn.pth", weights_only=True))
trained_net.eval()

state, _ = test_env.reset()
max_len = len(raw_data_test) - 1

confusion_matrix = [0, 0, 0, 0] # TP, FP, TN, FN
total_reward = 0

start_time = time()
for i in range(max_len):
    with torch.no_grad():
        action = trained_net(torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)).max(1).indices

    next_state, reward, terminated, _, info = test_env.step(action.item())

    reward = torch.tensor([reward], device=device)
    total_reward += reward.item()

    confusion_index = (0 if "P" in info["confusion"] else 1) if "T" in info["confusion"] else (2 if "P" in info["confusion"] else 3)
    confusion_matrix[confusion_index] += 1

    state = next_state
    end_time = time()
    loading_bar(i, max_len, end_time - start_time)

print()
test_env.close()

print(f"confusion matrix: {confusion_matrix}")

tp = confusion_matrix[0]
tn = confusion_matrix[1]
fp = confusion_matrix[2]
fn = confusion_matrix[3]

accuracy = (tp + tn) / (tp + fp + fn + tn)
print(f"Accuracy : {accuracy:.5}")
precision = tp / (tp + fp)
print(f"Precision: {precision:.5}")
recall = tp / (tp + fn) if tp + fn != 0 else 0.0
print(f" Recall  : {recall:.5}")
f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0.0
print(f"    F1   : {f1:.5}")
fpr = fp / (fp + tn) if fp + tn != 0 else 0.0
print(f"    FPR  : {fpr:.5}")

metrics = [accuracy, precision, recall, f1, fpr]
metrics = [n * 100 for n in metrics]
metrics_labels = ["Accuracy", "Precision", "Recall", "F1", "FPR"]

plt.figure(1)
plt.bar(metrics_labels, metrics)
plt.grid()
